In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from policy import Policy
from langchain_community.embeddings import OllamaEmbeddings

policy = Policy(data_dir='storage/policy', embedding=OllamaEmbeddings(model='llama3'))
policy_tools = policy.get_tools()
policy_tools

{'lookup_policy_tool': LookupPolicyTool(policy=<policy.Policy object at 0x00000239021603A0>)}

In [3]:
print(policy_tools['lookup_policy_tool'].invoke(input={'query': "حداکثر برای چند نفر میشه بلیط تهیه کرد؟"}))

FAQ_SUBJECT: قطار
FAQ_QUESTION:
در یک خرید، برای چند مسافر به‌صورت همزمان می‌توانم خرید کنم؟
FAQ_ANSWER:
<p>حداکثر بلیت قابل ارائه به هر کاربر غیر حضوری در هر خرید برای قطار های اتوبوسی و 4 تخته تعداد 4 بلیت ، قطار های 6 تخته تعداد 6 بلیت و واگن حمل خودرو تعداد 1 بلیت میباشد .</p><p><br>&nbsp;</p>

FAQ_SUBJECT: پرواز خارجی
FAQ_QUESTION:
خرید بلیط به‌صورت  گروهی چه شرایطی دارد؟
FAQ_ANSWER:
<p style="text-align:right;">برای مسافران بیشتر از 20 نفر می‌توانید با مرکز پشتیبانی تماس بگیرید و اطلاعات پرواز و مسافران خود را اعلام کنید تا درخواست خرید گروهی شما بررسی و بلیط‌ها تهیه شود.</p><ul><li style="text-align:justify;"><a href="tel:+982143900000">۰۲۱-۴۳۹۰۰۰۰۰ : شماره تماس</a></li></ul>

FAQ_SUBJECT: هتل داخلی
FAQ_QUESTION:
چه ساعتی می‌توانم اتاق را تحویل بگیرم و چه ساعتی باید اتاق را تحویل بدهم؟
FAQ_ANSWER:
<p style="text-align:right;">معمولا در اکثر هتل ها ساعت ورود 14 و خروج در ساعت 12 انجام میگردد .&nbsp;</p>

FAQ_SUBJECT: پرواز خارجی
FAQ_QUESTION:
چگونه می‌توانم در مورد مقررات پرواز ی

In [307]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model='llama3', num_ctx=8192, num_thread=8, temperature=0.0)

In [330]:
from online_search import PersianTavilySearchResults

tools = [PersianTavilySearchResults(max_results=3, llm=llm)] + list(policy_tools.values())

def get_tool_description(tool):
    tool_params = [
        f"{name}: {info['type']} ({info['description']})"
        for name, info in tool.args.items()
    ]
    tool_params_string = ', '.join(tool_params)
    return (
        f"tool_name -> {tool.name}\n"
        f"tool_params -> {tool_params_string}\n"
        f"tool_description ->\n{tool.description}"
    )

print(get_tool_description(tools[0]))

tool_name -> tavily_search_results_json
tool_params -> query: string (search query to look up)
tool_description ->
A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


In [357]:
import os
import json
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage


os.environ["TAVILY_API_KEY"] = "<>"


class ToolMessage(HumanMessage):
    """Ollama does not support `tool` role and `ToolMessage`"""


tools = [PersianTavilySearchResults(max_results=3, llm=llm)] + list(policy_tools.values())

tool_descs = '\n\n'.join([get_tool_description(tool) for tool in tools])

prompt_template = \
f"""
You are a helpful Persian customer support assistant for Iran Airlines.
Use the provided tools to search for flights, company policies, and other information to assist the user's queries. 
When searching, be persistent. Expand your query bounds if the first search returns no results. 
If a search comes up empty, expand your search before giving up.

You have access to the following tools to get more information if needed:

{tool_descs}

You also have access to the history of privious messages.

Generate the response in the following json format.
{{
    "THOUGHT": "<you should always think about what to do>",
    "ACTION": "<the action to take, must be one tool_name from above tools>",
    "ACTION_PARAMS": "<the input parameters to the ACTION, it must be in json format complying with the tool_params>"
    "FINAL_ANSWER": "<a text containing the final answer to the original input question>",
    "FINAL_ANSWER_PERSIAN": "<the Persian translated version of the FINAL_ANSWER>"
}}
If you don't know the answer, you can take an action using one of the provided tools.
But if you do, don't take and action and leave the action-related attributes empty.
The values `ACTION` and `FINAL_ANSWER` cannot be filled at the same time.

Always make sure that your output is a json complying with above format.
Do NOT add anything before or after the json response.
"""

user_question = "حداکثر برای چند نفر میتوان بلیط تهیه کرد؟"
messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "Let me check our policies on booking multiple tickets.", 
        "ACTION": "lookup_policy_tool", 
        "ACTION_PARAMS": {"query": "maximum number of passengers for a single booking"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[1].invoke(input={"query": "maximum number of passengers for a single booking"})
    ),
]

# print(messages[-1].content)
print(llm.invoke(messages).content)

{"THOUGHT": "I think I can find the answer to your question about booking multiple tickets.", 
"ACTION": "", 
"ACTION_PARAMS": {}, 
"FINAL_ANSWER": "According to our policies, you can book a maximum of 9 tickets for a single passenger. However, if you need to book more than one ticket for different passengers, please note that each passenger must have their own booking reference number.", 
"FINAL_ANSWER_PERSIAN": "بر اساس سیاست‌های ما، می‌توانید تا ۹ بلیط برای یک مسافر در یک زمان خریداری کنید. اما اگر بیش از یک بلیط برای مسافران مختلف نیاز دارید، لطفاً توجه داشته باشید که هر مسافر باید شماره مرجع خرید خود را داشته باشد."}


In [361]:
user_question = "هزینه اضافه بار برای مسافرت خارجی چقدر است؟"
messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "Let's check the child fare policy for Iran Airlines.", 
        "ACTION": "lookup_policy_tool", 
        "ACTION_PARAMS": {"query": "excess baggage fee for international flights"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[1].invoke(input={'query': "excess baggage fee for international flights"})
    ),
    AIMessage(json.dumps({
        "THOUGHT": "The provided tools didn't have the information about excess baggage fee for international flights.", 
        "ACTION": "tavily_search_results_json", 
        "ACTION_PARAMS": {"query": "excess baggage fee for international flights Iran Airlines"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[0].invoke(input={'query': "excess baggage fee for international flights Iran Airlines"})
    ),
    # AIMessage(json.dumps({
    #     "THOUGHT": "The search result suggests that Iran Airlines has a policy for excess baggage fees, but it's not explicitly stated.", 
    #     "ACTION": "lookup_policy_tool", 
    #     "ACTION_PARAMS": {"query": "excess baggage fee for international flights Iran Airlines"} 
    # })),
    # ToolMessage(
    #     "Here is the tool results:\n" +
    #     tools[1].invoke(input={'query': "excess baggage fee for international flights Iran Airlines"})
    # ),
]

# print(messages[-1].content)
print(llm.invoke(messages).content)

{"THOUGHT": "I found the information about excess baggage fee for international flights.",  "ACTION": "",  "ACTION_PARAMS": {},  "FINAL_ANSWER": "The cost of extra baggage on Iran Air international flights is 100,000 tomans plus 9% tax on added value from all Turkish airports to all Iranian airports (per kilogram of extra baggage): 9 euros. The purchase of extra baggage for adult and child passengers on international flights 'Hema' is possible in a package. Prices are categorized by destinations in Asia, near and far, Europe, and Baku.", "FINAL_ANSWER_ PERSIAN": ""}


In [353]:
user_question = "نرخ بلیط هواپیما برای کودکان زیر ۱۰ سال چقدر است؟"
messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "Let's check the child fare policy for Iran Airlines.", 
        "ACTION": "lookup_policy_tool", 
        "ACTION_PARAMS": {"query": "child fare policy iran airlines"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[1].invoke(input={'query': "child fare policy iran airlines"})
    ),
]

# print(messages[-1].content)
print(llm.invoke(messages).content)

{"THOUGHT": "The child fare policy for Iran Airlines is mentioned in the FAQ answer.", 
"FINAL_ANSWER": "According to the FAQ, the child fare for children under 10 years old is half of the adult ticket price.", 
"FINAL_ANSWER_PERSIAN": "بر اساس پاسخFAQ، نرخ بلیط کودک برای کودکان زیر ۱۰ سال نصف نرخ بلیط بزرگسال است."}


In [355]:
user_question = "نرخ بلیط هواپیما برای کودکان زیر ۱۰ سال چقدر است؟"
messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "Let's check the child fare policy for Iran Airlines.", 
        "ACTION": "lookup_policy_tool", 
        "ACTION_PARAMS": {"query": "child fare policy for children under 10 years old"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[1].invoke(input={'query': "child fare policy for children under 10 years old"})
    ),
]

# print(messages[-1].content)
print(llm.invoke(messages).content)

{"THOUGHT": "Unfortunately, the provided tools didn't have any information about child fare policy for children under 10 years old.", 
"FINAL_ANSWER": "We don't have specific information on child fare policy for children under 10 years old. However, we can suggest that you contact our customer support team or visit our website to get more information on this topic.", 
"FINAL_ANSWER_PERSIAN": "متاسفانه، ابزارهای ما اطلاعات خاصی درباره نرخ بلیط کودک برای کودکان زیر ۱۰ سال نداشتند. اما می‌توانیم پیشنهاد کنیم که با تیم پشتیبانی客رو تماس بگیرید یا به وب‌سایت ما مراجعه کنید تا اطلاعات بیشتر در مورد این موضوع بدست آورید."}
